In [1]:
import os, shutil

os.environ["CUDA_VISIBLE_DEVICES"]="-1"

log_dir = f'./logs/hyperparameter_tuning_BaseModel/'

try:
    # clearing logging directory
    shutil.rmtree(log_dir)
except NotADirectoryError:
    pass

if not os.path.exists(log_dir):
    os.makedirs(log_dir)


In [2]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

from KAGGLE_NLP_with_disaster_tweets.model import base_model, utils, constants
from KAGGLE_NLP_with_disaster_tweets.data_preparation.utils import DataPipeline, BatchPipeline

from tensorboard import notebook

2022-03-02 17:18:16.543595: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-02 17:18:16.543614: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
train_file_name = 'train.csv'
test_file_name = 'test.csv'
sample_submission_file_name = 'sample_submission.csv'
output_sequence_length = 30

In [4]:
max_vocabulary_size = 20000
glove_embedding_dim = 200

data_pipeline = DataPipeline(train_file_name, test_file_name, sample_submission_file_name,
                             output_sequence_length=30,
                             max_vocabulary_size=max_vocabulary_size,
                             glove_embedding_dim=glove_embedding_dim,
                             glove_url=constants.glove_url['twitter.27B'])
dataset = data_pipeline.prepare_dataset(include_cols=["location", "keyword"], apply_preprocessing=True)

2022-03-02 17:18:18.529469: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-02 17:18:18.529517: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: sersery-desktop
2022-03-02 17:18:18.529523: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: sersery-desktop
2022-03-02 17:18:18.529608: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.103.1
2022-03-02 17:18:18.529630: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.103.1
2022-03-02 17:18:18.529635: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.103.1
2022-03-02 17:18:18.530017: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU

Getting the file: ../data/train.csv
-----------------------------------------------------------------------------------------
Dataset 
Size: 7613
Dataset examples:
Input: [  15   37 3396    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0]
Target: [1. 0.]
Input: [   1   13   96 2948    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    

In [5]:
balanced_class_weights = utils.get_balanced_class_weights(data_pipeline.dataframe)

In [6]:
log_directory = "./logs/hyperparameter_tuning_BaseModel/"
utils.start_logging(log_directory, constants.hyperparameters, constants.metrics)

In [7]:
session_num = 1
hyperparameters = {"learning_rate": hp.HParam("learning_rate", hp.RealInterval(1e-5, 1e-3)),
                   "hidden_unit": hp.HParam("hidden_unit", hp.Discrete([16, 64])),
                   "batch_size": hp.HParam("batch_size", hp.Discrete([16, 32])),
                   "optimizer": hp.HParam("optimizer", hp.Discrete(["adamw"])),
                   "class_weights": hp.HParam("class_weights", hp.Discrete(["none", "balanced"])),
                   "dropout": hp.HParam("dropout", hp.Discrete([0.1, 0.4]))}
# look for the best hyperparameters
for optimizer in hyperparameters["optimizer"].domain.values:
    for hidden_unit in hyperparameters["hidden_unit"].domain.values:
        for batch_size in hyperparameters["batch_size"].domain.values:
            for learning_rate in (hyperparameters["learning_rate"].domain.min_value,
                                  hyperparameters["learning_rate"].domain.max_value):
                for dropout in hyperparameters["dropout"].domain.values:
                    for class_weights in hyperparameters["class_weights"].domain.values:
                        print(f"********************Session {session_num} started**********************")
                        hparams = {
                            hyperparameters["optimizer"]: optimizer,
                            hyperparameters["hidden_unit"]: hidden_unit,
                            hyperparameters["batch_size"]: batch_size,
                            hyperparameters["learning_rate"]: learning_rate,
                            hyperparameters["class_weights"]: class_weights,
                            hyperparameters["dropout"]: dropout,
                        }
                        model = base_model.BaseModel(
                            vocabulary_size=data_pipeline.vocabulary_size,
                            embedding_dim=64,
                            hidden_dim=128,
                            lstm_dim=64,
                            n_labels=2,
                            epochs=10,
                            batch_pipeline=BatchPipeline(dataset, batch_size),
                            hparam_manager=utils.HyperparameterManager(hparams),
                            embeddings_initializer=tf.keras.initializers.LecunUniform())
                            # embeddings_initializer=data_pipeline.build_embeddings_initializer())

                        with tf.summary.create_file_writer(f'{log_directory}{model.run_name}').as_default():
                            hp.hparams(hparams)
                            accuracy, precision, recall, f1, predictions = model.fit_and_evaluate(
                                class_weights=balanced_class_weights,
                                log_directory=log_directory)
                            tf.summary.scalar("accuracy", accuracy, step=1)
                            tf.summary.scalar("precision", precision, step=1)
                            tf.summary.scalar("recall", recall, step=1)
                            tf.summary.scalar("f1", f1, step=1)
                        session_num+=1

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          320000    
                                                                 
 bidirectional (Bidirectiona  (None, None, 64)         24832     
 l)                                                              
                                                                 
 dense (Dense)               (None, None, 16)          1040      
                                                                 
 dropout (Dropout)           (None, None, 16)          0         
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_1 (Dropout)         (None, 16)                0

/home/sersery/anaconda3/envs/tf-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 64)          320000    
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 64)         24832     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, None, 16)          1040      
                                                                 
 dropout_2 (Dropout)         (None, None, 16)          0         
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_3 (Dropout)         (None, 16)               

/home/sersery/anaconda3/envs/tf-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, None, 64)          320000    
                                                                 
 bidirectional_17 (Bidirecti  (None, None, 64)         24832     
 onal)                                                           
                                                                 
 dense_34 (Dense)            (None, None, 64)          4160      
                                                                 
 dropout_34 (Dropout)        (None, None, 64)          0         
                                                                 
 global_average_pooling1d_17  (None, 64)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dropout_35 (Dropout)        (None, 64)              

/home/sersery/anaconda3/envs/tf-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (None, None, 64)          320000    
                                                                 
 bidirectional_19 (Bidirecti  (None, None, 64)         24832     
 onal)                                                           
                                                                 
 dense_38 (Dense)            (None, None, 64)          4160      
                                                                 
 dropout_38 (Dropout)        (None, None, 64)          0         
                                                                 
 global_average_pooling1d_19  (None, 64)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dropout_39 (Dropout)        (None, 64)              

In [8]:
%load_ext tensorboard

In [9]:
tensorboard --logdir logs/hyperparameter_tuning_BaseModel --port 5000

Launching TensorBoard...